In [71]:
import pandas as pd
import numpy as np
import math

spambase = pd.read_csv('spambase.csv',header=None)
spambase.rename(columns={0:"word_freq_make", 1:"word_freq_address", 2:"word_freq_all", 3:"word_freq_3d", 4:"word_freq_our", 
                    5:"word_freq_over", 6:"word_freq_remove", 7:"word_freq_internet", 8:"word_freq_order", 9:"word_freq_mail",
                    10:"word_freq_receive", 11:"word_freq_will", 12:"word_freq_people", 13:"word_freq_report", 14:"word_freq_addresses",
                    15:"word_freq_free", 16:"word_freq_business", 17:"word_freq_email", 18:"word_freq_you", 19:"word_freq_credit", 
                    20:"word_freq_your", 21:"word_freq_font", 22:"word_freq_000", 23:"word_freq_money", 24:"word_freq_hp", 
                    25:"word_freq_hpl", 26:"word_freq_george", 27:"word_freq_650", 28:"word_freq_lab", 29:"word_freq_labs", 
                    30:"word_freq_telnet", 31:"word_freq_857", 32:"word_freq_data", 33:"word_freq_415", 34:"word_freq_85", 
                    35:"word_freq_technology", 36:"word_freq_1999", 37:"word_freq_parts", 38:"word_freq_pm", 39:"word_freq_direct", 
                    40:"word_freq_cs", 41:"word_freq_meeting", 42:"word_freq_original", 43:"word_freq_project", 44:"word_freq_re",
                    45:"word_freq_edu", 46:"word_freq_table", 47:"word_freq_conference", 48:"char_freq_;", 49:"char_freq_(", 
                    50:"char_freq_[", 51:"char_freq_!", 52:"char_freq_$", 53:"char_freq_#", 54:"capital_run_length_average", 
                    55:"capital_run_length_longest", 56:"capital_run_length_total", 57:"is_spam"},inplace=True)

class Lista():
    def __init__(self, k):
        self.lista = []
        self.num_neighbours = 5

    def add_pivo(self, pivo):
        self.pivo = pivo
    
    def adiciona(self, new):
        distance = new.calculate_distance(self.pivo)

        if len(self.lista) != 0 and distance > self.lista[-1].calculate_distance(self.pivo):
            return 
        if len(self.lista) == self.num_neighbours:
            self.lista.pop(-1)
        self.lista.append(new)
        self.lista.sort(key=lambda x: x.calculate_distance(self.pivo))

    def qtd_exemplares_por_especie(self):
        epe = {}
        for i in self.lista:
            if epe.get(i.spam) == None:
                epe[i.spam] = 1
            else:
                epe[i.spam] += 1
        return epe
        

    def define_spam(self):
        qtd_por_spam = self.qtd_exemplares_por_spam()
        #print("qtd_por_especie:", qtd_por_especie)
        maior = list(qtd_por_spam.keys())[0]

        for e in qtd_por_spam:
            if qtd_por_spam[e] > qtd_por_spam[maior]:
                maior = e
            elif  qtd_por_spam[e] == qtd_por_spam[maior]:
                maior = self.melhor_media(e, maior)  
        self.pivo.set_spam(maior)
        return True

    def melhor_media(self):
        menor = self.lista[0]
        menor_distancia = self.pivo.calculate_distance(menor)
        for i in self.lista:
            distancia_i = self.pivo.calculate_distance(i)
            if distancia_i < menor_distancia:
                menor = i
                menor_distancia = distancia_i
        return menor.specie
        

class Node ():
    def __init__(self, dataArr):
        self.dataArr = dataArr

    def set_spam(self, spam):
        self.spam = spam
    
    def calculate_distance(self, other_node):
        #try creating and traversing an array you dumb fuck
        dA = np.subtract(self.dataArr, other_node.dataArr)
        
        dA = dA * dA
        
        return math.sqrt(dA)
    
    def show(self):
        print("Mail Characteristics:", self.dataArr,
        "is Spam:", self.spam)
        

class KNN ():
    def __init__(self, k, tamanho_treino):
        self.k = k
        self.tamanho_treino = tamanho_treino
        self.distance_matrix = []
        self.nodes = []
        self.training_population = None
        self.test_population = None

    def main (self):
        self.set_populations(self.tamanho_treino)
        self.training_algorithm()
        self.test_algorithm()
    
    def set_populations(self, training_population_size):
        spam = pd.read_csv("spambase.csv")
        spam = spam.sample(frac=1).reset_index(drop=True)
        self.training_population = spam.iloc[:training_population_size]
        self.test_population = spam.iloc[training_population_size:]
        
    def training_algorithm(self):
        self.nodes = self.make_nodes(self.training_population)
        
    def make_nodes(self, df):
        new_nodes = []
        mailData = []
        for row in df.iterrows():
            new_node = Node(mailData.append(row != "is_spam"))
            new_node.set_spam(df["is_spam"])
            new_nodes.append(new_node)
        return new_nodes        

    def test_algorithm(self):
        test_nodes = self.make_nodes(self.test_population)
        i = 0
        for node in test_nodes:
            minha_lista = Lista(self.k)

            minha_lista.add_pivo(node)
            for node in self.nodes:
                minha_lista.adiciona(node)
            minha_lista.define_spam()
            
            if minha_lista:
                self.nodes.append(minha_lista.pivo)
            if minha_lista.pivo.spam != self.test_population.get(["is_spam"]).iloc[i,0]:
                print("DIFERENTE")
            i += 1
        

if __name__ == "__main__":
    knn = KNN(7, 70)
    knn.main()

DIFERENTE
DIFERENTE
DIFERENTE
DIFERENTE
DIFERENTE
